
Word Level *RNN*

**References**

https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/

https://github.com/VeereshElango/text_generation_thirukkural/blob/master/Char_LSTM_Tamil.py

https://veereshelango.github.io/articles/2017-07/text-generation-with-thirukkural

https://github.com/SudarAbisheck/thirukkural-dataset # DATASET

https://github.com/vlraik/word-level-rnn-keras/blob/master/lstm_text_generation.py #WORD LEVEL RNN

https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/

https://medium.com/coinmonks/word-level-lstm-text-generator-creating-automatic-song-lyrics-with-neural-networks-b8a1617104fb

https://github.com/sachinruk/deepschool.io/blob/master/DL-Keras_Tensorflow/Lesson%2019%20-%20Seq2Seq%20-%20Date%20translator%20-%20Solutions.ipynb #DATE TRANSLATOR

https://medium.com/phrasee/neural-text-generation-generating-text-using-conditional-language-models-a37b69c7cd4b

https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html


In [0]:
!pip install Open-Tamil

Import the neccesary packages

In [0]:
import numpy as np
import pandas as pd
import tamil
import sys,os,io
from collections import Counter
from sklearn.model_selection import train_test_split
from keras.callbacks import LambdaCallback
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Activation

Using TensorFlow backend.


In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/KarthikMgk/kural_analysis_py/master/kural_dat_utf8.txt',index_col=0,sep=',',header='infer')

In [0]:
df.head()

,kural
1,அகர முதல எழுத்தெல்லாம் ஆதி பகவன் முதற்றே உலகு
2,கற்றதனால் ஆய பயனென்கொல் வாலறிவன் நற்றாள் தொழாஅ...
3,மலர்மிசை ஏகினான் மாணடி சேர்ந்தார் நிலமிசை நீடு...
4,வேண்டுதல் வேண்டாமை இலானடி சேர்ந்தார்க்கு யாண்ட...
5,இருள்சேர் இருவினையும் சேரா இறைவன் பொருள்சேர் ப...


Lets split the sentences into words

In [0]:
text_words = []

for i in df['kural'].values:
  text = [i for i in i.split(' ') if i.strip() != '' or i == '\n']
  for j in text:
    text_words.append(j)

In [0]:
chars = set(text_words)
print(len(chars), ' is the number of unique words')
print(len(text_words), ' is the total number of words')

6374  is the number of unique words
9391  is the total number of words


In [0]:
word_to_int = dict((c, i) for i, c in enumerate(text_words))
int_to_word = dict((i,c) for i, c in enumerate(text_words))

In [0]:
step = 1
max_len = 3

sentences = []
next_words = []

for i in range(0, len(text_words) - max_len, step):

  sentence = ' '.join(text_words[i:i+max_len])
  next_word = text_words[i + max_len]

  sentences.append(sentence)
  next_words.append(next_word)

  # sentences.append([word_to_int[i] for i in sentence])
  # next_words.append(word_to_int[next_word])

In [0]:
word_to_int['அச்சம்']

1034

In [0]:
X = np.zeros((len(sentences), max_len, len(text_words)), dtype=np.bool)
y = np.zeros((len(sentences), len(text_words)), dtype=np.bool)

for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence.split()):

        X[i, t, word_to_int[word]] = 1
  y[i, word_to_int[next_words[i]]] = 1

In [0]:
print('Build model...')
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=False, input_shape=(max_len, len(text_words)))))
# model.add(Dropout(0.5))
# model.add(LSTM(256, return_sequences=False))
# model.add(Dropout(0.2))
model.add(Dense(len(text_words)))
#model.add(Dense(1000))
model.add(Activation('softmax'))

Build model...



In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
# X = X.astype(int)
# y = y.astype(int)

https://github.com/llSourcell/How-to-Generate-Music-Demo/issues/4

In [0]:
x_train,x_val, y_train, y_val = train_test_split(X,y, test_size=0.2)

In [0]:
def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    a = np.log(a) / temperature
    dist = np.exp(a)/np.sum(np.exp(a))
    choices = range(len(a))
    return np.random.choice(choices, p=dist)

# train the model, output generated text after each iteration
for iteration in range(1, 15):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    hist = model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val,y_val))
    model.save_weights('GoTweights',overwrite=True)

    start_index = np.random.randint(0, len(sentences) - max_len - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)
        generated = ''
        sentence = sentences[start_index: start_index + 1]
        generated += ' '.join(sentence)
        print('----- Generating with seed: "' , sentence , '"')
        print()
        sys.stdout.write(generated)
        print()

        for i in range(7):
            x = np.zeros((1, max_len, len(text_words)))
            for t, word in enumerate(sentence):

                for w in (word.split(' ')):
                    x[0, t, word_to_int[w]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = int_to_word[next_index]
            generated += next_word
            del sentence[0]
            sentence.append(next_word)
            sys.stdout.write(' ')
            sys.stdout.write(next_word)
            sys.stdout.flush()
        print()


--------------------------------------------------
Iteration 1



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 7510 samples, validate on 1878 samples
Epoch 1/5
7510/7510 [==============================] - 10s 1ms/step - loss: 9.1401 - val_loss: 9.1461
Epoch 2/5
7510/7510 [==============================] - 8s 1ms/step - loss: 8.6182 - val_loss: 9.6485
Epoch 3/5
7510/7510 [==============================] - 8s 1ms/step - loss: 8.1803 - val_loss: 10.3315
Epoch 4/5
7510/7510 [==============================] - 8s 1ms/step - loss: 7.7937 - val_loss: 10.8225
Epoch 5/5
7510/7510 [==============================] - 8s 1ms/step - loss: 7.3952 - val_loss: 11.6219

----- diversity: 0.2
----- Generating with seed: " ['அதனைப் பலரறியார் பாக்கியத்'] "

அதனைப் பலரறியார் பாக்கியத்
 கொளலும் உள்ளதூஉம் தலை மாணப் உள்ளதூஉம் இன்னாது உறும்

----- diversity: 0.5
----- Generating with seed: " ['அதனைப் பலரறியார் பாக்கியத்'] "

அதனைப் பலரறியார் பாக்கியத்
 வ